<a href="https://colab.research.google.com/github/dani-lbnl/Leadership/blob/master/Matching_problem_alumna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-Problem" data-toc-modified-id="Define-Problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define Problem</a></span></li><li><span><a href="#Parameter" data-toc-modified-id="Parameter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameter</a></span></li><li><span><a href="#Decision-Variables" data-toc-modified-id="Decision-Variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Decision Variables</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Contraints" data-toc-modified-id="Contraints-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Contraints</a></span></li><li><span><a href="#Solve-the-Problem" data-toc-modified-id="Solve-the-Problem-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Solve the Problem</a></span></li></ul></div>

# Define Problem

In [13]:
#!pip install pulp
from pulp import *


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Parameter

In [14]:
import numpy as np 
import pandas as pd
np.random.seed(0)

c = np.random.randint(0,10, (4,4))
np.fill_diagonal(c,0)

c

array([[0, 0, 3, 3],
       [7, 0, 3, 5],
       [2, 4, 0, 6],
       [8, 8, 1, 0]])

In [118]:
mentors = range(4)
mentees = range(4)

ngroups = len(mentors)

In [139]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Matching_people", LpMaximize)
prob

Matching_people:
MAXIMIZE
None
VARIABLES

# Decision Variables

\begin{equation} \label{y_var_def}
    y_{i,j} = \left \{ \begin{array}{ll}
      1 & \mbox{if employee $i$ is paired with employee $j$}\\
      0 & \mbox{otherwise.} \end{array} \right.
  \end{equation}

In [140]:
#Define the variables
y = LpVariable.dicts("pair", [(i,j) for i in mentees for j in mentors] ,cat='Binary')

In [141]:
namesMentors = ['Larissa','Renata','Gisele','Daniela']
namesMentees = ['Baby','Bia','Caca','Didi']

match_info = pd.DataFrame(c, index=namesMentees, columns=namesMentors)

match_info

,Larissa,Renata,Gisele,Daniela
Baby,0,0,3,3
Bia,7,0,3,5
Caca,2,4,0,6
Didi,7,8,1,0


# Objective

Maximize the preference scores between mentors $i$ and mentees $j$

Maximize $\sum_{j=0}^n\sum_{i=0}^n (c_{i,j}+c_{j,i}) \cdot y_{i,j}$

In [142]:
prob += lpSum([(c[i][j]) * y[(i,j)] for i in mentees for j in mentors])
prob

Matching_people:
MAXIMIZE
3*pair_(0,_2) + 3*pair_(0,_3) + 7*pair_(1,_0) + 3*pair_(1,_2) + 5*pair_(1,_3) + 2*pair_(2,_0) + 4*pair_(2,_1) + 6*pair_(2,_3) + 7*pair_(3,_0) + 8*pair_(3,_1) + 1*pair_(3,_2) + 0
VARIABLES
0 <= pair_(0,_2) <= 1 Integer
0 <= pair_(0,_3) <= 1 Integer
0 <= pair_(1,_0) <= 1 Integer
0 <= pair_(1,_2) <= 1 Integer
0 <= pair_(1,_3) <= 1 Integer
0 <= pair_(2,_0) <= 1 Integer
0 <= pair_(2,_1) <= 1 Integer
0 <= pair_(2,_3) <= 1 Integer
0 <= pair_(3,_0) <= 1 Integer
0 <= pair_(3,_1) <= 1 Integer
0 <= pair_(3,_2) <= 1 Integer

# Contraints

Employee $i$ is paired with no more than one employee $j$

\begin{equation}
\sum_{i} y_{i,j}\leq 1 \; \forall j \;\in employees
\end{equation}

Employee $j$ is paired with no more than one employee $i$

\begin{equation}
\sum_{i} y_{j,i} \leq 1 \; \forall j \;\in employees
\end{equation}

Pairing between employee $i$ and $j$ also means to pair between employee $j$ and $i$

\begin{equation}
y_{i,j} + y_{j,i} \leq 1 \; \forall i,j \;\in employees
\end{equation}

There is a total of 4 pairs

\begin{equation}
\sum_j\sum_{i} y_{i,j} = 4
\end{equation}

In [143]:
for i in mentees:
 prob += lpSum(y[(i,j)] for j in mentees) <= 1
 prob += lpSum(y[(j,i)] for j in mentors) <= 1
    #prob += lpSum(y[(i,j)] for j in mentors)+ lpSum(y[(j,i)] for j in mentees) <= 1


prob += lpSum(y[(i,j)] for i in mentors for j in mentees) == ngroups

# Solve the Problem

In [144]:
prob.solve()

1

In [145]:
print("Finish matching!\n")
for i in mentees:
    for j in mentors:
        #print(i,j,'=',y[(i,j)].varValue)
        if y[(i,j)].varValue == 1:
            print('Mentor {} & mentee {} are a match. Total score: {}'.format(namesMentors[j],namesMentees[i], c[i,j] ))

Finish matching!

Mentor Gisele & mentee Baby are a match. Total score: 3
Mentor Larissa & mentee Bia are a match. Total score: 7
Mentor Daniela & mentee Caca are a match. Total score: 6
Mentor Renata & mentee Didi are a match. Total score: 8


In [80]:
pulp.value(prob.objective)

24.0

In [108]:
c[3,0] = 7.4

In [90]:
y

{(0, 0): pair_(0,_0),
 (0, 1): pair_(0,_1),
 (0, 2): pair_(0,_2),
 (0, 3): pair_(0,_3),
 (1, 0): pair_(1,_0),
 (1, 1): pair_(1,_1),
 (1, 2): pair_(1,_2),
 (1, 3): pair_(1,_3),
 (2, 0): pair_(2,_0),
 (2, 1): pair_(2,_1),
 (2, 2): pair_(2,_2),
 (2, 3): pair_(2,_3),
 (3, 0): pair_(3,_0),
 (3, 1): pair_(3,_1),
 (3, 2): pair_(3,_2),
 (3, 3): pair_(3,_3)}

In [98]:
for 
y[(0,0)].varValue

AttributeError: ignored